In [1]:
%load_ext autoreload
%autoreload 2

import re
import pickle
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

import rdkit
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem

from utils.features import *

Loaded 97 descriptor functions


In [37]:
df = pd.read_csv("dataset/test/AID_1259354/AID_1259354_datatable_all.csv", skiprows=[1,2,3], header=0, index_col=0)
df = df[(~df.PUBCHEM_EXT_DATASOURCE_SMILES.isna())&(~df.PUBCHEM_ACTIVITY_OUTCOME.isna())]
print((df.PUBCHEM_ACTIVITY_OUTCOME=='Active').sum())
print((df.PUBCHEM_ACTIVITY_OUTCOME=='Inactive').sum())
df

1823
54542


,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,% Inhibition
PUBCHEM_RESULT_TAG,,,,,,,,
2,348401744,694901.0,C1=CC=C2C(=C1)C(=O)C(C2=O)C3=CC=NC=C3,Active,NaN,NaN,NaN,99.0
3,348401745,46344002.0,CC(C)C1=CC=C(C=C1)NC(=O)C2=CC3=C(C=C2)N4CCCCC4...,Active,NaN,NaN,NaN,99.0
4,348401746,3772003.0,C1CC(CN(C1)CC2=CC3=C(C=C2)C4=CC=CC=C4C3)CO,Active,NaN,NaN,NaN,98.0
5,348401747,764882.0,CC1=CC=CN2C1=NC(=C(C2=O)C=NC3=CC=C(C=C3)OC)NC,Active,NaN,NaN,NaN,98.0
6,348401748,131841611.0,CC1(CC(CC(N1)(C)C)N2C(=O)OC(C2(C)O)(C)[C@H]3CC...,Active,NaN,NaN,NaN,98.0
...,...,...,...,...,...,...,...,...
77696,348479438,755180.0,C1CN(CCN1C2=CC=CC=C2)C3=NC=NC(=C3)N4C=CC=N4,Inactive,NaN,NaN,NaN,NaN
77697,348479439,705269.0,CCOC(=O)C1=CC=C(C=C1)OCC2=C(ON=C2C)C,Inactive,NaN,NaN,NaN,NaN
77698,348479440,2873589.0,C1=CC=C(C=C1)N2C=C(C(=N2)C3=CC=C(C=C3)Br)C(=O)O,Inactive,NaN,NaN,NaN,NaN


In [38]:
smiles_ls = df.PUBCHEM_EXT_DATASOURCE_SMILES.tolist()
mol_ls = list(map(lambda x: rdkit.Chem.MolFromSmiles(x), smiles_ls))
fp_ls = [AllChem.GetMorganFingerprintAsBitVect(x, radius=2, nBits = 1024) for x in mol_ls]

activity_ls = df.PUBCHEM_ACTIVITY_OUTCOME
activity_ls = activity_ls.map({'Inactive':0, 'Active':1}).tolist()

ds_ls = get_descriptors_parallel(mol_ls)

[14:49:13] WARNING: not removing hydrogen atom without neighbors


In [39]:
def get_scaffold(mol: rdkit.Chem.rdchem.Mol) -> str:
    return Chem.MolToSmiles(MakeScaffoldGeneric(GetScaffoldForMol(mol)))

In [40]:
from collections import defaultdict
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles, MakeScaffoldGeneric, GetScaffoldForMol

scaffold_dict = defaultdict(list)
for idx, mol in zip(range(len(mol_ls)), mol_ls):
    scaffold_dict[get_scaffold(mol)].append(idx)

In [41]:
data = {
    "smiles_ls": smiles_ls,
    "mol_ls": mol_ls,
    "fp_ls": fp_ls,
    "activity_ls": activity_ls,
    "ds_ls": ds_ls,
    "scaffold_dict": scaffold_dict
}
with open("dataset/test/AID_1259354/data.pkl", "wb") as file:
    pickle.dump(data, file)

In [42]:
def preprocess_pubchem(path: str):
    df = pd.read_csv(path, skiprows=[1,2,], header=0, index_col=0)
    df = df[(~df.PUBCHEM_EXT_DATASOURCE_SMILES.isna())&(~df.PUBCHEM_ACTIVITY_OUTCOME.isna())]
    
    smiles_ls = df.PUBCHEM_EXT_DATASOURCE_SMILES.tolist()
    mol_ls = list(map(lambda x: rdkit.Chem.MolFromSmiles(x), smiles_ls))
    fp_ls = [AllChem.GetMorganFingerprintAsBitVect(x, radius=2, nBits = 1024) for x in mol_ls]
    activity_ls = df.PUBCHEM_ACTIVITY_OUTCOME
    activity_ls = activity_ls.map({'Inactive':0, 'Active':1}).tolist()
    ds_ls = get_descriptors_parallel(mol_ls)
    
    scaffold_dict = defaultdict(list)
    for idx, mol in zip(range(len(mol_ls)), mol_ls):
        scaffold_dict[get_scaffold(mol)].append(idx)
        

    data = {
        "smiles_ls": smiles_ls,
        "mol_ls": mol_ls,
        "fp_ls": fp_ls,
        "activity_ls": activity_ls,
        "ds_ls": ds_ls,
        "scaffold_dict": scaffold_dict
    }
    with open(os.path.dirname(path)+"/data.pkl", "wb") as file:
        pickle.dump(data, file)

In [43]:
preprocess_pubchem("dataset/test/AID_488969/AID_488969_datatable_all.csv")
preprocess_pubchem("dataset/test/AID_598/AID_598_datatable_all.csv")

C:\Users\25791\AppData\Local\Temp\ipykernel_16980\2143785741.py:2: DtypeWarning: Columns (4,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, skiprows=[1,2,], header=0, index_col=0)
